In [21]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.transforms
import seaborn as sns
import math
from math import sqrt, factorial
import re
from itertools import groupby
import itertools
import scipy
import random
from decimal import Decimal
from operator import itemgetter
import multiprocessing
from CustomFunctions import PCvisualization, DetailedBalance, segment_LLS, linear_cycle_utils
from pathlib import Path
from aicsimageio.writers.ome_tiff_writer import OmeTiffWriter
from aicsimageio.readers.czi_reader import CziReader
from CustomFunctions.MO_Threshold import MO_ma
import skimage
from scipy.spatial import distance
import numpy.ma as ma

In [4]:
#get directories and open separated datasets

whichpcs = [2,7]
time_interval = 5 #sec/frame

datadir = 'E:/Aaron/random_lls/'
savedir = datadir + 'random_longtracks/'
if not os.path.exists(savedir):
    os.makedirs(savedir)

TotalFrame = pd.read_csv(datadir + 'Shape_Metrics_transitionPCbins.csv', index_col=0)

centers = pd.read_csv(datadir+'PC_bin_centers.csv', index_col=0)

nbins = np.max(TotalFrame[[x for x in TotalFrame.columns.to_list() if 'bin' in x]].to_numpy())

#restrict dataframe to only random experiments
TotalFrame = TotalFrame[TotalFrame.Treatment=='Random']


#list of behavioral stats to track with CGPS transitions
behaviorstats = ['speed','persistence','Turn_Angle','directional_autocorrelation','time']

#find the length of cell consecutive frames
results = []
for i, cells in TotalFrame.groupby('CellID'):
    cells = cells.sort_values('time').reset_index(drop = True)
    runs = list()
    #######https://stackoverflow.com/questions/2361945/detecting-consecutive-integers-in-a-list
    for k, g in groupby(enumerate(cells['frame']), lambda ix: ix[0] - ix[1]):
        currentrun = list(map(itemgetter(1), g))
        list.append(runs, currentrun)
    maxrun = max([len(l) for l in runs])
    sumrun = sum([len(l) for l in runs])
    actualrun = max(runs, key=len, default=[])
    results.append([i, maxrun, actualrun, sumrun])
#find
stdf = pd.DataFrame(results, columns = ['CellID','length_of_run','actual_run','total_runs']).sort_values('total_runs', ascending=False).reset_index(drop=True)
#get cells with more than 30 minutes of continuous footage
longcells = stdf[stdf.total_runs>360].CellID.to_list()
TotalFrame = TotalFrame[TotalFrame.CellID.isin(longcells)].copy()
stdf

,CellID,length_of_run,actual_run,total_runs
0,20240520_488_EGFP-CAAX_561_mysoin-mApple_37C_c...,73,"[164, 165, 166, 167, 168, 169, 170, 171, 172, ...",618
1,20240611_488_EGFP-CAAX_640_actin-halotag_cell7_01,42,"[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 5...",546
2,20240524_488_EGFP-CAAX_640_actin-halotag_cell1_01,40,"[14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 2...",543
3,20240524_488_EGFP-CAAX_640_actin-halotag_cell2_01,40,"[43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 5...",529
4,20240520_488_EGFP-CAAX_561_mysoin-mApple_37C_c...,36,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",529
...,...,...,...,...
97,20240527_488_EGFP-CAAX_640_SPY650-DNA_cell3_06,4,"[4, 5, 6, 7]",4
98,20240520_488_EGFP-CAAX_561_mysoin-mApple_37C_c...,2,"[1, 2]",3
99,20240805_488_EGFP-CAAX_640_SPY650-DNA_cell2_02,2,"[1, 2]",2
100,20240524_488_EGFP-CAAX_640_actin-halotag_cell2_06,1,[1],1


In [37]:
############ define the cell to make a continuous movie of and some other info
ddir = '//10.158.28.35/TheriotLab_LLS7/Aaron/manual_crop_lls_decon/'
xy_buffer = 7
z_buffer = 7

cellname = '20240805_488_EGFP-CAAX_640_SPY650-DNA_cell1_01'
cell = TotalFrame[TotalFrame.CellID==cellname]
cell = cell.sort_values('time').reset_index(drop=True)
lst = []
for x in cell.cell.to_list():
    x = x.split('_frame')[0]
    if x not in lst:
        lst.append(x)


In [ ]:
####### loop through all of the movies that captured the cell of interest
####### MODIFIED FROM segment_LLS.getbb_movie
centroids = []
imagelist = []
framelist = []
for l in lst:
    big = CziReader(ddir+l+'_deskewed_decon.czi')
    bigim = big.data
    orig_shape = big.shape
    cents = []
    reimlist = []
    croplist = []
    ### start by rescaling all of the images
    with multiprocessing.Pool(processes=60) as pool: 
        rescaled = pool.map(segment_LLS.quarter_scale, [i for i in bigim[:,1,:,:,:]])
    with multiprocessing.Pool(processes=60) as pool: 
        rescaledother = pool.map(segment_LLS.quarter_scale, [i for i in bigim[:,0,:,:,:]])
    ### get the shape of the rescaled to use later to leave out objects
    shape = np.array(rescaled).shape    
    #loop through time points to get bounding boxes
    cropdf = pd.DataFrame()
    for it, f in enumerate(rescaled):
        #mask the top pixels with signal
        if (f>100).any():
            mare = ma.masked_array(f, mask = f>np.percentile(f[f>100], 97))
            seg = MO_ma(mare, global_thresh_method='tri', object_minArea=200, local_adjust = 0.92)
            im_labeled, n_labels = skimage.measure.label(
                                      seg, background=0, return_num=True,  )
        
            im_props = skimage.measure.regionprops(im_labeled)
            tempdf = pd.DataFrame([])
            for count, prop in enumerate(im_props):
                z,y,x = np.array(prop.centroid)
                thebox = np.array(prop.bbox)
                area = prop.area * 64
                td = {'cell':count, 'z_min':thebox[0], 'y_min':thebox[1], 
                        'x_min':thebox[2],'z_max':thebox[3], 'y_max':thebox[4], 'x_max':thebox[5],
                       'z':z, 'y':y, 'x': x, 'z_range': seg.shape[-3], 'area':area}
                #ensure only things that aren't on the edge are chosen
                #and are big enough
                if (td['z_min']>0) and (td['y_min']>0) and (td['x_min']>0) and (td['z_max']/4<shape[-3]) and (td['y_max']/4<shape[-2]) and (td['x_max']/4<shape[-1]) and (area>50000):
                    tempdf = tempdf.append(td, ignore_index=True)
    
    
            if len(tempdf)>0:
                #if there's only one option pick that
                if len(tempdf)==1:
                    cropdf = cropdf.append(tempdf, ignore_index=True)
                #if first frame get the closest object to the center
                elif (it==0):
                    dists = []
                    for p, row in tempdf.iterrows():
                        dists.append(distance.pdist([np.array(shape)[-3:][::-1]/2,
                                        row[['x','y','z']].values]))
                    cropdf = cropdf.append(tempdf.iloc[np.argmin(dists)], ignore_index=True)
                #if it's any other frame get the closest object to the last pick
                else:
                    dists = []
                    for p, row in tempdf.iterrows():
                        dists.append(distance.pdist([cropdf[['x','y','z']].iloc[-1].values,
                                        row[['x','y','z']].values]))
                    cropdf = cropdf.append(tempdf.iloc[np.argmin(dists)], ignore_index=True)
            #if there's no options fill the gap with nan
            else:
                cropdf = cropdf.append({'cell':np.nan, 'z_min':np.nan, 'y_min':np.nan,
                        'x_min':np.nan,'z_max':np.nan, 'y_max':np.nan, 'x_max':np.nan,
                       'z':np.nan, 'y':np.nan, 'x': np.nan, 'z_range': np.nan, 'area':np.nan}, ignore_index=True)
        else:
            cropdf = cropdf.append({'cell':np.nan, 'z_min':np.nan, 'y_min':np.nan, 
                    'x_min':np.nan,'z_max':np.nan, 'y_max':np.nan, 'x_max':np.nan,
                   'z':np.nan, 'y':np.nan, 'x': np.nan, 'z_range': np.nan, 'area':np.nan}, ignore_index=True)


    ###aggregate cropping info from this particular image
    cropdf['x_min'] = int(max(0, cropdf['x_min'].min()-xy_buffer))
    cropdf['y_min'] = int(max(0, cropdf['y_min'].min()-xy_buffer))
    cropdf['z_min'] = int(max(0, cropdf['z_min'].min()-z_buffer))
    cropdf['z_max'] = int(min(cropdf['z_max'].max()+z_buffer, shape[-3]))
    cropdf['y_max'] = int(min(cropdf['y_max'].max()+xy_buffer, shape[-2])+1)
    cropdf['x_max'] = int(min(cropdf['x_max'].max()+xy_buffer, shape[-1])+1)
    mincrop = np.min(cropdf[['z_min','y_min','x_min']].values, axis = 0)
    maxcrop = np.max(cropdf[['z_max','y_max','x_max']].values, axis = 0)
    rescaledall = np.stack((rescaled,rescaledother))
    imagelist.append(rescaledall[:,:,
                            mincrop[0]:maxcrop[0],
                          mincrop[1]:maxcrop[1],
                          mincrop[2]:maxcrop[2]])
    centroids.append(cropdf[['z','y','x']].values-np.array([mincrop[0],mincrop[1],mincrop[2]]))
    framelist.append([l+'_frame_'+'{:03d}'.format(r) for r in range(len(cropdf))])
    print(f'finished cropping move {l}')
###pull all the images into one movie with minimal dimensions
newc = []
for u, c in enumerate(centroids):
    if u!=0:
        newc.append(centroids[u-1][-1]-c[0])
    else:
        newc.append(np.zeros(3))
zeropos = np.stack(newc)
zeropos = np.cumsum(zeropos,axis=0)
zeropos = np.round(zeropos + abs(zeropos.min())+0.0001)
shapes = [x.shape for x in imagelist]
totalshape = np.round(np.max(zeropos + np.array(shapes)[:,-3:], axis = 0)-np.min(zeropos,axis=0)+0.001)
finalimage = np.zeros((np.concatenate([[sum([x[1]for x in shapes])],[2],totalshape]).astype(int)))
timecount = 0
shifts = zeropos-np.min(zeropos,axis=0)
for r in range(len(shapes)):
    s = shapes[r]
    z = shifts[r].astype(int)
    finalimage[timecount:timecount+s[1],
               :,
               z[-3]:z[-3]+s[-3],
               z[-2]:z[-2]+s[-2],
               z[-1]:z[-1]+s[-1],
              ] = np.swapaxes(imagelist[r],0,1)
    timecount = timecount+s[1]
    
indivdir = savedir + 'singlecells/'+ cellname +'/'
if not os.path.exists(indivdir):
    os.makedirs(indivdir)
OmeTiffWriter.save(finalimage.astype('uint16'),indivdir+f'{cellname}_full_movie.ome.tiff', dim_order='TCZYX')
pd.Series(framelist).to_csv(indivdir+f'{cellname}_framelist.csv')

finished cropping move 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell1-01-Subset-01
finished cropping move 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell1-02-Subset-01
finished cropping move 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell1-03-Subset-01
finished cropping move 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell1-04-Subset-01
finished cropping move 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell1-05-Subset-01
finished cropping move 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell1-06-Subset-01
finished cropping move 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell1-07-Subset-01
finished cropping move 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell1-08-Subset-01
finished cropping move 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell1-09-Subset-01


In [33]:
indivdir = savedir + 'singlecells/'+ cellname +'/'
if not os.path.exists(indivdir):
    os.makedirs(indivdir)
OmeTiffWriter.save(finalimage.astype('uint16'),indivdir+f'{cellname}_full_movie.ome.tiff'
pd.Series(framelist).to_csv(indivdir+f'{cellname}_framelist.csv')